In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam

In [2]:
# model parameters
reuse = False
block = schnet_block
head = cat3Head
data = '../Data/{}_data_ACSF_expand.pickle'
batch_size = 32
dim = 128
epochs = 20
clip = 2
layer1 = 2
layer2 = 2
factor = 2
lr = 1e-4

#### Train base

In [3]:
train_dl,val_dl = get_data(data,batch_size)

In [3]:
model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

In [11]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr)

In [6]:
model,train_loss_perType,val_loss_perType = train(opt,model,epochs,train_dl,val_dl,paras,clip)

epoch:0, train_loss: +0.815, val_loss: +0.211, 
train_vector: +2.90|+1.43|+0.52|-0.01|+0.14|+0.83|+0.65|+0.06, 
val_vector  : +1.35|+0.72|+0.20|-0.67|-0.48|+0.69|+0.38|-0.50

epoch:1, train_loss: +0.061, val_loss: +0.032, 
train_vector: +0.98|+0.45|+0.07|-0.73|-0.60|+0.62|+0.33|-0.61, 
val_vector  : +1.03|+0.31|+0.05|-0.59|-0.79|+0.66|+0.28|-0.70

epoch:2, train_loss: -0.124, val_loss: -0.141, 
train_vector: +0.71|+0.18|-0.09|-0.96|-0.82|+0.55|+0.24|-0.79, 
val_vector  : +0.65|+0.28|-0.17|-0.58|-1.08|+0.54|+0.16|-0.92

epoch:3, train_loss: -0.230, val_loss: -0.261, 
train_vector: +0.59|+0.04|-0.19|-1.10|-0.95|+0.51|+0.17|-0.91, 
val_vector  : +0.50|+0.35|-0.27|-1.23|-1.05|+0.49|+0.11|-0.99

epoch:4, train_loss: -0.305, val_loss: -0.385, 
train_vector: +0.49|-0.07|-0.25|-1.18|-1.05|+0.48|+0.13|-0.99, 
val_vector  : +0.38|-0.27|-0.31|-1.16|-1.22|+0.48|+0.07|-1.05

epoch:5, train_loss: -0.358, val_loss: -0.424, 
train_vector: +0.44|-0.12|-0.30|-1.26|-1.12|+0.46|+0.09|-1.05, 
val_vector  :

In [8]:
save_results(train_loss_perType,val_loss_perType,reuse,block,\
             head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

In [13]:
save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

#### train on each type

In [3]:
model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

In [6]:
lr_type =1e-4/2
epochs_type = 10

In [4]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=lr_type)

In [9]:
file_name = !ls ../Model/
file_name = file_name[1]

In [11]:
for i in range(8):
    data_type = '../Data/{}_data_ACSF_expand_type_'+str(i)+'.pickle'
    train_dl,val_dl = get_data(data_type,batch_size)

    checkpoint = torch.load('../Model/'+file_name)
    model.load_state_dict(checkpoint['model_state_dict'])
    opt.load_state_dict(checkpoint['optimizer_state_dict'])

    for para in opt.param_groups:
        para['lr'] = lr_type
    
    print('\nTraining type: {}'.format(i))
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True)

    save_results(train_loss_perType,val_loss_perType,reuse,block,\
                 head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i))

    save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i))


Training type: 0
epoch:0, train_loss: +0.069, val_loss: -0.064, 
train_vector: +0.07|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.06|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: -0.026, val_loss: +0.310, 
train_vector: -0.03|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.31|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -0.072, val_loss: -0.131, 
train_vector: -0.07|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : -0.13|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -0.105, val_loss: +0.163, 
train_vector: -0.11|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.16|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -0.132, val_loss: +0.020, 
train_vector: -0.13|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.02|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:5, train_loss: -0.155, val_loss: +0.028, 
train_vector: -0.16|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.

epoch:6, train_loss: -1.854, val_loss: -1.846, 
train_vector: +0.00|+0.00|+0.00|+0.00|-1.85|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|-1.85|+0.00|+0.00|+0.00

epoch:7, train_loss: -1.874, val_loss: -1.881, 
train_vector: +0.00|+0.00|+0.00|+0.00|-1.87|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|-1.88|+0.00|+0.00|+0.00

epoch:8, train_loss: -1.894, val_loss: -1.953, 
train_vector: +0.00|+0.00|+0.00|+0.00|-1.89|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|-1.95|+0.00|+0.00|+0.00

epoch:9, train_loss: -1.909, val_loss: -1.859, 
train_vector: +0.00|+0.00|+0.00|+0.00|-1.91|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|-1.86|+0.00|+0.00|+0.00

Training completed in 189.78208541870117s

Training type: 5
epoch:0, train_loss: +0.124, val_loss: +0.023, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.12|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.02|+0.00|+0.00

epoch:1, train_loss: -0.016, val_loss: -0.092, 
train_vector: +0.

#### Train on each type from start

In [9]:
head = cat3Head_type

In [8]:
class cat3Head_type(torch.nn.Module):
    def __init__(self,dim,edge_in3,edge_in4):
        cat_factor = 3
        super(cat3Head_type, self).__init__()
        self.linear = Sequential(#BatchNorm1d(dim*cat_factor),
                                 Linear(dim*cat_factor,dim*cat_factor*2),
                                 ReLU(inplace=True),
                                 #BatchNorm1d(dim*cat_factor*2),
                                 Linear(dim*cat_factor*2,1))
        
    def forward(self,x,edge_index3,edge_attr3,edge_attr4):
        temp = x[edge_index3] # (2,N,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)
        yhat = self.linear(yhat).squeeze(1)
        return yhat

In [6]:
lr_type =1e-4
epochs_type = 25

In [10]:
for i in range(8):
    # load type data
    data_type = '../Data/{}_data_ACSF_expand_type_'+str(i)+'.pickle'
    train_dl,val_dl = get_data(data_type,batch_size)
    
    # init model
    model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type)
    
    print('\nTraining type: {}'.format(i))
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True)

    save_results(train_loss_perType,val_loss_perType,reuse,block,\
                 head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')

    save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_start')


Training type: 0
epoch:0, train_loss: +0.955, val_loss: +0.476, 
train_vector: +0.95|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.48|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.467, val_loss: +0.587, 
train_vector: +0.47|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.59|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: +0.325, val_loss: +0.178, 
train_vector: +0.32|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.18|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: +0.212, val_loss: +0.232, 
train_vector: +0.21|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.23|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: +0.143, val_loss: +0.036, 
train_vector: +0.14|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.04|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:5, train_loss: +0.095, val_loss: -0.003, 
train_vector: +0.10|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.

epoch:22, train_loss: -0.674, val_loss: -0.502, 
train_vector: +0.00|-0.67|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|-0.50|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:23, train_loss: -0.650, val_loss: -0.788, 
train_vector: +0.00|-0.65|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|-0.79|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:24, train_loss: -0.708, val_loss: -0.561, 
train_vector: +0.00|-0.71|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|-0.56|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

Training completed in 333.60658383369446s

Training type: 2
epoch:0, train_loss: -0.235, val_loss: -0.207, 
train_vector: +0.00|+0.00|-0.24|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|-0.21|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: -0.680, val_loss: -0.749, 
train_vector: +0.00|+0.00|-0.68|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|-0.75|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -0.843, val_loss: -0.908, 
train_vector: 

epoch:18, train_loss: -1.995, val_loss: -2.031, 
train_vector: +0.00|+0.00|+0.00|-1.99|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-2.03|+0.00|+0.00|+0.00|+0.00

epoch:19, train_loss: -2.012, val_loss: -1.796, 
train_vector: +0.00|+0.00|+0.00|-2.01|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.80|+0.00|+0.00|+0.00|+0.00

epoch:20, train_loss: -2.022, val_loss: -1.650, 
train_vector: +0.00|+0.00|+0.00|-2.02|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.65|+0.00|+0.00|+0.00|+0.00

epoch:21, train_loss: -2.038, val_loss: -1.849, 
train_vector: +0.00|+0.00|+0.00|-2.04|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.85|+0.00|+0.00|+0.00|+0.00

epoch:22, train_loss: -2.077, val_loss: -1.475, 
train_vector: +0.00|+0.00|+0.00|-2.08|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.47|+0.00|+0.00|+0.00|+0.00

epoch:23, train_loss: -2.065, val_loss: -1.962, 
train_vector: +0.00|+0.00|+0.00|-2.07|+0.00|+0.00|+0.00|+0.00, 
val_vec

epoch:14, train_loss: -1.017, val_loss: -0.991, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.02|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-0.99|+0.00|+0.00

epoch:15, train_loss: -1.034, val_loss: -1.007, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.03|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.01|+0.00|+0.00

epoch:16, train_loss: -1.052, val_loss: -1.041, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.05|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.04|+0.00|+0.00

epoch:17, train_loss: -1.070, val_loss: -1.052, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.07|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.05|+0.00|+0.00

epoch:18, train_loss: -1.086, val_loss: -1.039, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.09|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|-1.04|+0.00|+0.00

epoch:19, train_loss: -1.100, val_loss: -1.085, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|-1.10|+0.00|+0.00, 
val_vec

epoch:10, train_loss: -1.999, val_loss: -1.705, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-1.70

epoch:11, train_loss: -2.031, val_loss: -1.955, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.03, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-1.96

epoch:12, train_loss: -2.072, val_loss: -1.972, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.07, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-1.97

epoch:13, train_loss: -2.096, val_loss: -1.947, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.10, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-1.95

epoch:14, train_loss: -2.128, val_loss: -1.994, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.13, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-1.99

epoch:15, train_loss: -2.156, val_loss: -2.026, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-2.16, 
val_vec

#### two stage bigger head

In [3]:
lr_type =1e-4/2
epochs_type = 10
file_name = "False_<class 'functions_refactor.schnet_block'>_<class 'functions_refactor.cat3Head'>__data_ACSF_expand.pickle_32_128_2_2_2_2_20_base.tar"

In [4]:
for i in range(8):
    # load type data
    data_type = '../Data/{}_data_ACSF_expand_type_'+str(i)+'.pickle'
    train_dl,val_dl = get_data(data_type,batch_size)
    
    # load base model
    model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    checkpoint = torch.load('../Model/'+file_name)
    model.load_state_dict(checkpoint['model_state_dict'])

    set_requires_grad(model, False)
    model.head = cat3Head_type(dim,8,data_dict[data]['edge_in4']).to('cuda:0')
    
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type) 
    # train head
    print('\nTraining type: {}'.format(i))
    print('\nTraining head')
    model,train_loss_perType,val_loss_perType = train(opt,model,5,train_dl,val_dl,paras,clip,True)
    
    # train all
    set_requires_grad(model, True)
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr_type)    
    print('\nTraining all')
    model,train_loss_perType,val_loss_perType = train(opt,model,epochs_type,train_dl,val_dl,paras,clip,True,train_loss_perType,val_loss_perType)
    
    save_results(train_loss_perType,val_loss_perType,reuse,block,\
                 head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_2stage')

    save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'type_'+str(i)+'_2stage')


Training type: 0

Training head
epoch:0, train_loss: +0.579, val_loss: +0.257, 
train_vector: +0.58|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.26|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: +0.008, val_loss: +0.116, 
train_vector: +0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.12|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -0.002, val_loss: +0.046, 
train_vector: -0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.05|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -0.009, val_loss: +0.129, 
train_vector: -0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.13|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -0.014, val_loss: +0.143, 
train_vector: -0.01|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.14|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00|+0.00

Training completed in 84.75621557235718s

Training all
epoch:5, train_loss: +0.044, val_loss


Training type: 3

Training head
epoch:0, train_loss: -1.243, val_loss: -1.555, 
train_vector: +0.00|+0.00|+0.00|-1.24|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.55|+0.00|+0.00|+0.00|+0.00

epoch:1, train_loss: -1.708, val_loss: -1.742, 
train_vector: +0.00|+0.00|+0.00|-1.71|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.74|+0.00|+0.00|+0.00|+0.00

epoch:2, train_loss: -1.738, val_loss: -1.674, 
train_vector: +0.00|+0.00|+0.00|-1.74|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.67|+0.00|+0.00|+0.00|+0.00

epoch:3, train_loss: -1.754, val_loss: -1.776, 
train_vector: +0.00|+0.00|+0.00|-1.75|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.78|+0.00|+0.00|+0.00|+0.00

epoch:4, train_loss: -1.764, val_loss: -1.685, 
train_vector: +0.00|+0.00|+0.00|-1.76|+0.00|+0.00|+0.00|+0.00, 
val_vector  : +0.00|+0.00|+0.00|-1.69|+0.00|+0.00|+0.00|+0.00

Training completed in 86.44345188140869s

Training all
epoch:5, train_loss: -1.658, val_loss


Training type: 6

Training head
epoch:0, train_loss: -0.196, val_loss: -0.409, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.20|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.41|+0.00

epoch:1, train_loss: -0.426, val_loss: -0.449, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.43|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.45|+0.00

epoch:2, train_loss: -0.449, val_loss: -0.470, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.45|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.47|+0.00

epoch:3, train_loss: -0.475, val_loss: -0.497, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.48|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.50|+0.00

epoch:4, train_loss: -0.492, val_loss: -0.504, 
train_vector: +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.49|+0.00, 
val_vector  : +0.00|+0.00|+0.00|+0.00|+0.00|+0.00|-0.50|+0.00

Training completed in 83.67579579353333s

Training all
epoch:5, train_loss: -0.620, val_loss

### make submissions

In [14]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

From start

In [13]:
head = cat3Head_type

In [ ]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs,'_start')

In [ ]:
head = cat3Head_type

In [ ]:
make_submission(reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)